## Gera Arquivo de EUSD, Kei, Meta e Realizado para Cálculo de Compensação por Continuidade - Pós Operação

In [ ]:
import pandas as pd
import os
import datetime
import pyarrow

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)
referencia_calculo = '2023_08'

### importa eusd e kei (fio b):

In [ ]:
%%time
pasta_fonte = r'C:/Users/evgj/Downloads/BI_REN1000/BASES/EUSD_KEY'

os.chdir(pasta_fonte)
filepaths = [f for f in os.listdir('.') if f.startswith(referencia_calculo)]

df_eusd_kei_colunas_texto = {'NUMCDC_CAD': 'str', 'NUMANO_REF': 'str', 'NUMMES_REF': 'str'}

df_eusd_kei_0 = pd.concat(
        (pd.read_csv(i, \
            dtype = df_eusd_kei_colunas_texto, \
            encoding = 'cp1252', sep = ';', engine='python', decimal = ',', thousands = '.') for i in filepaths)
).reset_index(drop = True)

df_eusd_kei_0.shape


In [ ]:
df_eusd_kei_0.rename(columns = {'NUMANO_REF': 'NUMANO_REF_EUSD', 'NUMMES_REF': 'NUMMES_REF_EUSD'}, inplace = True)

In [ ]:
df_eusd_kei_0 = df_eusd_kei_0[['EMPRESA', 'NUMCDC_CAD', 'NUMANO_REF_EUSD', 'NUMMES_REF_EUSD', 'VLRENC_DTB', 'IDTTIP_FOR', 'K', 'EUSD_FIO_B']].copy()


In [ ]:
df_eusd_kei_0

### importa "result dic" do SiQual:
### !!! IMPORTAR APENAS METAS

In [ ]:
%%time
pasta_fonte = r'C:/Users/evgj/Downloads/BI_REN1000/BASES/EUSD_KEY/RESULTADOS_DIC'

os.chdir(pasta_fonte)
filepaths = [f for f in os.listdir('.') if f.startswith(referencia_calculo)]

df_result_dic_colunas_importar = ['RESULTADO', 'ENTIDADE', 'META_DIC_MENSAL', 'META_FIC_MENSAL', 'META_DMIC_MENSAL', 'META_DICRI_HORA']
df_result_dic_colunas_texto = {'ENTIDADE': 'str'}


df_result_dic_0 = pd.concat(
        (pd.read_csv(i, \
            dtype = df_result_dic_colunas_texto, \
            usecols = df_result_dic_colunas_importar, \
            encoding = 'cp1252', sep = ';', engine='python', decimal = ',', thousands = '.') for i in filepaths)
).reset_index(drop = True)

df_result_dic_0.shape


In [ ]:
df_result_dic_0.head()

#### proporção de UCs da base result dic contidas na base eusd kei:

In [ ]:
df_result_dic_0.ENTIDADE.isin(df_eusd_kei_0.NUMCDC_CAD).sum() / df_result_dic_0.shape[0]

#### proporção de UCs da base eusd kei contidas na base result dic:

In [ ]:
df_eusd_kei_0.NUMCDC_CAD.isin(df_result_dic_0.ENTIDADE).sum() / df_eusd_kei_0.shape[0]

### importa dic dec bruto (obs.: alterar o mês)

In [ ]:
%%time
pasta_fonte = r'C:/Users/evgj/Downloads/BI_REN1000/BASES/EUSD_KEY/DIC_FIC_BR'

os.chdir(pasta_fonte)
filepaths = [f for f in os.listdir('.') if f.startswith(referencia_calculo)]

df_dic_dec_br_colunas_importar = ['RESULTADO', 'CONTA', 'DIC_MES', 'FIC_MES', 'DMIC_MES']
df_dic_dec_br_colunas_texto = {'CONTA': 'str'}


df_dic_dec_br_0 = pd.concat(
        (pd.read_csv(i, \
            dtype = df_dic_dec_br_colunas_texto, \
            usecols = df_dic_dec_br_colunas_importar, \
            encoding = 'cp1252', sep = ';', engine='python', decimal = ',', thousands = '.') for i in filepaths)
).reset_index(drop = True)

df_dic_dec_br_0.shape



#### proporção de UCs da base dic dec br contidas na base eusd kei:

In [ ]:
df_dic_dec_br_0.CONTA.isin(df_eusd_kei_0.NUMCDC_CAD).sum() / df_dic_dec_br_0.shape[0]

#### proporção de UCs da base dic dec br contidas na base result dic:

In [ ]:
df_dic_dec_br_0.CONTA.isin(df_result_dic_0.ENTIDADE).sum() / df_dic_dec_br_0.shape[0]

### criando cópias para manter as bases originais antes das transformações:

In [ ]:
df_eusd_kei = df_eusd_kei_0.copy()
df_result_dic = df_result_dic_0.copy()
df_dic_dec_br = df_dic_dec_br_0.copy()

### removendo duplicatas da base de dic dec bruto:

In [ ]:
df_dic_dec_br = df_dic_dec_br.drop_duplicates(['RESULTADO', 'CONTA'])

### adicionando coluna de "mês referência":

In [ ]:
df_result_dic['mes_ref_carta'] = referencia_calculo.replace('_', '-')
df_dic_dec_br['mes_ref_carta'] = referencia_calculo.replace('_', '-')

### fazendo join das bases:

In [ ]:
eusd_kei_meta_real = df_eusd_kei.merge(
        df_result_dic[['ENTIDADE', 'META_DIC_MENSAL', 'META_FIC_MENSAL','META_DMIC_MENSAL', 'META_DICRI_HORA', 'mes_ref_carta']], 
        how = 'left', 
        left_on = 'NUMCDC_CAD', 
        right_on = 'ENTIDADE'
    ).merge(
    df_dic_dec_br[['RESULTADO', 'CONTA', 'DIC_MES', 'FIC_MES', 'DMIC_MES', 'mes_ref_carta']], 
    how = 'left', 
    left_on = ['mes_ref_carta', 'NUMCDC_CAD'], 
    right_on = ['mes_ref_carta', 'CONTA']
)

In [ ]:
#eusd_kei_meta_real['mes_ref_carta'] = eusd_kei_meta_real['mes_ref_carta'].replace('VPR-DIC','2023-08')
eusd_kei_meta_real['mes_ref_carta'].value_counts()


In [ ]:
eusd_kei_meta_real.to_parquet(r'C:/Users/evgj/Downloads/BI_REN1000/BASES/eusd_kei_meta_real.parquet')


___

In [ ]:
valida = eusd_kei_meta_real.copy()
valida['mes_ref_carta'] = valida['mes_ref_carta'].replace('VPR-DIC','2023-08')
valida.head()

In [ ]:
eusd_kei_meta_real.META_DICRI_HORA.value_counts()

In [ ]:
eusd_kei_meta_real['EMPRESA'].unique()

In [ ]:
eusd_kei_meta_real.head()